In [58]:
import os

In [59]:
%pwd

'/Users/vishalchandru/MacBook/KidneyCNN'

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [61]:
from src.Kidney_CTScan.utils.utility import read_yaml,create_directories
from src.Kidney_CTScan.constants import *

In [62]:
class ConfigurationManager():
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
                                    root_dir = config.root_dir,
                                    source_url = config.source_url,
                                    local_data_file =  config.local_data_file,
                                    unzip_dir = config.unzip_dir
                                    )
        return data_ingestion_config

In [63]:
import os
import zipfile
import sys
import gdown
from src.Kidney_CTScan import logger
from src.Kidney_CTScan.utils.utility import get_size
from src.Kidney_CTScan import CustomException

In [64]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_data(self) -> str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f'Downloading dataset from {dataset_url} to local directory {zip_download_dir}')

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f'Downloaded dataset from {dataset_url} to local directory {zip_download_dir}')

        except Exception as e:
            raise CustomException(e,sys)
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as f:
            f.extractall(unzip_path)

In [65]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)

[ 2023-11-22 20:43:06,137 ] INFO : utility - 20 - Read the yaml file from config/config.yaml
[ 2023-11-22 20:43:06,139 ] INFO : utility - 20 - Read the yaml file from params.yaml
[ 2023-11-22 20:43:06,139 ] INFO : utility - 31 - Created a directory on artifacts
[ 2023-11-22 20:43:06,140 ] INFO : utility - 31 - Created a directory on artifacts/data_ingestion
[ 2023-11-22 20:43:06,140 ] INFO : 3603411225 - 10 - Downloading dataset from https://drive.google.com/file/d/1k2QcmjA_35NwiUBV68lZPUXy-UUGEvJK/view?usp=share_link to local directory artifacts/data_ingestion/data.zip


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1k2QcmjA_35NwiUBV68lZPUXy-UUGEvJK
From (redirected): https://drive.google.com/uc?/export=download&id=1k2QcmjA_35NwiUBV68lZPUXy-UUGEvJK&confirm=t&uuid=398fea47-e823-4ce0-8a47-886cc864a34f
To: /Users/vishalchandru/MacBook/KidneyCNN/artifacts/data_ingestion/data.zip
100%|██████████| 943M/943M [01:00<00:00, 15.5MB/s] 

[ 2023-11-22 20:44:07,426 ] INFO : 3603411225 - 15 - Downloaded dataset from https://drive.google.com/file/d/1k2QcmjA_35NwiUBV68lZPUXy-UUGEvJK/view?usp=share_link to local directory artifacts/data_ingestion/data.zip
